# Titanic ML Competition

...

### Introduction

### Objectives

### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Importing the dataset

In [ ]:
titanic_df = pd.read_csv("Datasets/train.csv")
titanic_df.head()

In [ ]:
titanic_df.info()

#### Renaming of the columns & changing index

In [ ]:
renamed_columns = {"Pclass":"Economic status","SibSp":"Number of siblings/spouses","Parch":"Number of parents/children"}
titanic_df.rename(columns=renamed_columns,inplace = True)

In [ ]:
titanic_df.set_index(titanic_df["PassengerId"],inplace = True)
titanic_df.drop(columns="PassengerId",inplace=True)

#### Data Types

In [ ]:
titanic_df.select_dtypes(include=['object']).nunique()

In [ ]:
titanic_df["Sex"] = titanic_df["Sex"].astype("category")
titanic_df["Embarked"] = titanic_df["Embarked"].astype("category")

In [ ]:
titanic_df["Economic status"] = titanic_df["Economic status"].astype("category")

In [ ]:
titanic_df[["Number of siblings/spouses","Number of parents/children"]].max()

In [ ]:
titanic_df["Number of siblings/spouses"] = titanic_df["Number of siblings/spouses"].astype("int8")
titanic_df["Number of parents/children"] = titanic_df["Number of parents/children"].astype("int8")

In [ ]:
titanic_df["Survived"] = titanic_df["Survived"].astype("int8")

#### Missing values

In [ ]:
titanic_df["Cabin"].fillna("-",inplace=True)

In [ ]:
titanic_df.drop(columns=["Ticket"],inplace=True)

#### Data conversions

In [ ]:
new_economic_status_names = {1:"Upper",2:"Middle",3:"Lower"}
titanic_df["Economic status"].cat.rename_categories(new_economic_status_names,inplace=True)

In [ ]:
new_port_names = {"C":"Cherbourg","Q":"Queenstown","S":"Southhampton"}
titanic_df["Embarked"].cat.rename_categories(new_port_names,inplace = True)

In [ ]:
new_sex = {"male":"Male","female":"Female"}
titanic_df["Sex"].cat.rename_categories(new_sex,inplace = True)

#### Reordering columns

In [ ]:
personal_info = ["Name","Sex","Age"]
economic_status = ["Economic status","Fare"]
family = ["Number of siblings/spouses","Number of parents/children"]
journey = ["Cabin","Embarked","Survived"]
new_order = personal_info + economic_status + family + journey
titanic_df = titanic_df.reindex(columns = new_order)

#### Dataset after handling it

In [ ]:
titanic_df.info()

In [ ]:
titanic_df.head()

### Exploratory Data Analysis